通过使用深度学习框架来简洁地实现 线性回归模型 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2,-3.4]) # 这里定义了2个权重
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size=10
data_iter=load_array((features,labels),batch_size)

next(iter(data_iter))

[tensor([[-0.3611, -0.4042],
         [ 0.9339,  1.4154],
         [-0.2837,  0.5076],
         [-1.1524,  0.5245],
         [ 0.2299, -0.8701],
         [-0.2935, -0.0057],
         [ 1.4649,  0.5509],
         [ 1.9312, -0.1023],
         [-0.7528,  0.0781],
         [-0.8626, -0.1243]]),
 tensor([[4.8342],
         [1.2542],
         [1.8953],
         [0.1069],
         [7.6066],
         [3.6378],
         [5.2558],
         [8.4204],
         [2.4261],
         [2.8906]])]

使用框架的预定义好的层

In [3]:
#'nn'是神经网络的缩写
from torch import nn
net = nn.Sequential(nn.Linear(2,1)) #2是输入，1是输出

初始化模型参数

```
net[0].weight：访问第一层（线性层）的权重矩阵
.data：获取参数的实际数值（张量数据）
.normal_(0, 0.01)：用正态分布初始化权重
均值 = 0
标准差 = 0.01
下划线 _ 表示就地操作（直接修改原数据）

net[0].bias：访问第一层的偏置向量
.data：获取偏置的实际数值
.fill_(0)：将偏置填充为0

为什么要手动初始化？
默认初始化的问题：
PyTorch会自动初始化参数，但可能不是最优的
不同的初始化策略会影响训练效果

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是MSELoss类，也称为平方L2范数

In [5]:
loss = nn.MSELoss()

实例化SGD实例

In [6]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

训练过程代码与我们从零开始时所做的非常相似

In [13]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)    
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000260
epoch 2, loss 0.000108
epoch 3, loss 0.000109
